In [142]:
#Loading the data set - training data.
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from gensim.models import KeyedVectors
from sklearn.preprocessing import Imputer
from nltk.tokenize import wordpunct_tokenize
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegressionCV
from sklearn.svm import LinearSVC, NuSVC
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

In [143]:
#dataset=pd.read_csv('datasets/tow_classes_tweet.csv', encoding='utf-8', usecols=['sentiment', 'txt'])
dataset=pd.read_csv('datasets/Ar_tweet_cleaned_final.csv', encoding='utf-8', usecols=['sentiment', 'txt'])


In [144]:
dataset.sentiment.value_counts()

1    993
0    958
Name: sentiment, dtype: int64

In [145]:
twenty_train, twenty_test = train_test_split(dataset, test_size=0.1)
twenty_train.head()
#prints all the categories

,sentiment,txt
392,1,اسال اله عزوجل له الثبات
161,0,﻿ واله ما يحر عزبت بنقاليه
851,1,﻿صباح الوفاء لك يا صديقي
905,1,﻿قصه جميله جدا تعكس معني الايمان و التمسك بالع...
174,0,وقعنا ولا ما وقعنا يلي بدهم يعملوه بيعملوه


In [146]:
len(twenty_train)

1755

In [147]:
len(twenty_test)

196

In [148]:
twenty_train.sentiment.value_counts()

1    899
0    856
Name: sentiment, dtype: int64

In [149]:
test.sentiment.value_counts()

0    109
1     87
Name: sentiment, dtype: int64

In [150]:
# NLTK
# Removing stop words
def get_stop_words(path):
    #"stop_words.txt"
    stop_words = []
    with codecs.open(path, "r", encoding="utf-8", errors="ignore") as myfile:
        stop_words = myfile.readlines()
    stop_words = [word.strip() for word in stop_words]
    return stop_words
stop_words = get_stop_words('Arabic_stop_word.txt')

from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer(stop_words=stop_words)), ('tfidf', TfidfTransformer()), 
                     ('clf', MultinomialNB())])

In [151]:
    # Extracting features from text files
    count_vect = CountVectorizer(stop_words=stop_words)
    X_train_counts = count_vect.fit_transform(twenty_train['txt'])
    print('X_train_counts.shape', X_train_counts.shape)
    # TF-IDF
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    print ('X_train_tfidf.shape',X_train_tfidf.shape)
    
    # Training Naive Bayes (NB) classifier on training data.
    clf = MultinomialNB().fit(X_train_tfidf, twenty_train.sentiment)

X_train_counts.shape (1755, 5656)
X_train_tfidf.shape (1755, 5656)


In [152]:
# Building a pipeline: We can write less code and do all of the above, by building a pipeline as follows:
# The names ‘vect’ , ‘tfidf’ and ‘clf’ are arbitrary but will be used later.
# We will be using the 'text_clf' going forward.
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

text_clf = text_clf.fit(twenty_train['txt'], twenty_train.sentiment)
predicted = text_clf.predict(twenty_test['txt'])
np.mean(predicted == twenty_test.sentiment)

0.8418367346938775

In [153]:
        y_pred = predicted
        y_test = twenty_test.sentiment
        f1_score = metrics.f1_score(y_test, y_pred)

        # macro accuracy (macro average)
        macc = metrics.f1_score(y_test, y_pred, pos_label=None, average='macro')

        # precision and recall
        recall = metrics.recall_score(y_test, y_pred)
        precision = metrics.precision_score(y_test, y_pred)
        accu=metrics.accuracy_score(y_test, y_pred)
        results = [accu,macc, f1_score, precision, recall]
        print("NB",results)

NB [0.8418367346938775, 0.8417337396785705, 0.845771144278607, 0.794392523364486, 0.9042553191489362]


In [154]:
# Enhancedddddddddddd
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 3))), ('tfidf', TfidfTransformer(use_idf=False)), ('clf', MultinomialNB(alpha=1e-2))])
text_clf = text_clf.fit(twenty_train['txt'], twenty_train.sentiment)
# Performance of NB Classifier
predicted = text_clf.predict(twenty_test['txt'])
print(np.mean(predicted == twenty_test.sentiment))
def accuracy():
        y_pred = predicted
        y_test = twenty_test.sentiment
        f1_score = metrics.f1_score(y_test, y_pred)

        # macro accuracy (macro average)
        macc = metrics.f1_score(y_test, y_pred, pos_label=None, average='macro')

        # precision and recall
        recall = metrics.recall_score(y_test, y_pred)
        precision = metrics.precision_score(y_test, y_pred)
        accu=metrics.accuracy_score(y_test, y_pred)

        results = [accu, macc, f1_score, precision, recall]
        print("Enhanced NB",results)
accuracy()

0.8673469387755102
Enhanced NB [0.8673469387755102, 0.8673469387755102, 0.8673469387755102, 0.8333333333333334, 0.9042553191489362]


In [155]:
parameters = {'vect__ngram_range': [(1, 1), (1, 3)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-4)}
# Next, we create an instance of the grid search by passing the classifier, parameters 
# and n_jobs=-1 which tells to use multiple cores from user machine.
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(twenty_train.txt, twenty_train.sentiment)
# To see the best mean score and the params, run the following code
gs_clf.best_score_
gs_clf.best_params_
# Output for above should be: The accuracy has now increased to ~90.6% for the NB classifier (not so naive anymore! 😄)
# and the corresponding parameters are {‘clf__alpha’: 0.01, ‘tfidf__use_idf’: True, ‘vect__ngram_range’: (1, 2)}.

{'clf__alpha': 0.01, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}

In [156]:
# Training Support Vector Machines - SVM and calculating its performance
text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42))])

text_clf_svm = text_clf_svm.fit(twenty_train.txt, twenty_train.sentiment)
predicted_svm = text_clf_svm.predict(twenty_test.txt)
print(np.mean(predicted_svm == twenty_test.sentiment))
def accuracy1():
        y_pred = predicted_svm
        y_test = twenty_test.sentiment
        f1_score = metrics.f1_score(y_test, y_pred)

        # macro accuracy (macro average)
        macc = metrics.f1_score(y_test, y_pred, pos_label=None, average='macro')

        # precision and recall
        recall = metrics.recall_score(y_test, y_pred)
        precision = metrics.precision_score(y_test, y_pred)
        accu=metrics.accuracy_score(y_test, y_pred)
        results = [accu, macc, f1_score, precision, recall]
        print(" SVM",results)
accuracy1()      

0.826530612244898
 SVM [0.826530612244898, 0.8262411347517731, 0.8191489361702128, 0.8191489361702128, 0.8191489361702128]


C:\Users\aas3n17\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [175]:
# Enhanceddddddddddddd
text_clf_svm = Pipeline([('vect', CountVectorizer(ngram_range=(1, 3))), ('tfidf', TfidfTransformer(use_idf=True)),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-4, n_iter=5, random_state=42))])
text_clf_svm = text_clf_svm.fit(twenty_train.txt, twenty_train.sentiment)
predicted_svm = text_clf_svm.predict(twenty_test.txt)
np.mean(predicted_svm == twenty_test.sentiment)

C:\Users\aas3n17\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.8418367346938775

In [176]:
        y_pred = predicted_svm
        y_test = twenty_test.sentiment
        f1_score = metrics.f1_score(y_test, y_pred)

        # macro accuracy (macro average)
        macc = metrics.f1_score(y_test, y_pred, pos_label=None, average='macro')

        # precision and recall
        recall = metrics.recall_score(y_test, y_pred)
        precision = metrics.precision_score(y_test, y_pred)
        accu=metrics.accuracy_score(y_test, y_pred)
        results = [accu,macc, f1_score, precision, recall]
        print("Enhanced SVM",results)

Enhanced SVM [0.8418367346938775, 0.8418326174671353, 0.841025641025641, 0.8118811881188119, 0.8723404255319149]


In [85]:
# Similarly doing grid search for SVM
from sklearn.model_selection import GridSearchCV
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False),'clf-svm__alpha': (1e-2, 1e-3)}
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(twenty_train.txt, twenty_train.sentiment)
gs_clf_svm.best_score_
gs_clf_svm.best_params_

C:\Users\aas3n17\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


{'clf-svm__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}

In [193]:
#NBSVM
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
train=twenty_train
test=twenty_test
import re
import string
re_tok = re.compile(r'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()
n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,3), tokenizer=tokenize, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(train['txt'])
test_term_doc = vec.transform(test['txt'])
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)
x = trn_term_doc
test_x = test_term_doc
def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=3, dual=True)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

label_cols=['sentiment']
preds = np.zeros((len(test), len(label_cols)))

for i, j in enumerate(label_cols):
    print('fit', j)
    m,r = get_mdl(train[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]
    

y_test_non_category = test.sentiment
y_predict_non_category = preds.round()

#print('\n Testing accuracy is {}'.format(accuracy_score(y_test_non_category, y_predict_non_category)))
#print ('\n F1',f1_score(y_test_non_category,y_predict_non_category, average='weighted'))
#print ('\n Recall:', recall_score(y_test_non_category, y_predict_non_category,average='weighted'))
#print ('\n Precision:', precision_score(y_test_non_category, y_predict_non_category, average='weighted'))
#print ('\n clasification report:\n', classification_report(y_test_non_category, y_predict_non_category))
#print ('\n confussion matrix:\n',confusion_matrix(y_test_non_category, y_predict_non_category))

fit sentiment


In [196]:
        y_pred = y_predict_non_category
        y_test = y_test_non_category
        f1_score = metrics.f1_score(y_test, y_pred)

        # macro accuracy (macro average)
        macc = metrics.f1_score(y_test, y_pred, pos_label=None, average='macro')

        # precision and recall
        recall = metrics.recall_score(y_test, y_pred)
        precision = metrics.precision_score(y_test, y_pred)
        accu=metrics.accuracy_score(y_test, y_pred)


        results = [accu, macc, f1_score, precision, recall]
        print("NB-SVM",results)

NB-SVM [0.8469387755102041, 0.846683354192741, 0.8404255319148938, 0.8404255319148937, 0.8404255319148937]


In [161]:
import argparse
from logging import info, basicConfig, INFO 
LOG_HEAD = '[%(asctime)s] %(levelname)s: %(message)s'
basicConfig(format=LOG_HEAD, level=INFO)
class ArSentiment(object):
    def __init__(self, train, test, embeddings_file=None, dataset_file=None, plot_roc=False, split=0.9, detailed=False):
        """
        :param embeddings_file: path to the embeddings file.
        :param dataset_file: path to a labeled dataset file.
        :param plot_roc: boolean, plot ROC figure.
        :param split: float, data split fraction i.e. train | test split (default: 90% | 10%)
        :param detailed: boolean, output classifiers' parameters info i.e. name, parameters' value, .. etc.
        """

        self.dataset_file = dataset_file
        self.split = split

        self.embeddings, self.dimension = self.load_vectors(embeddings_file)

        # read dataset
        train, test = train, test
        train_txt, test_txt = train['txt'], test['txt']
        self.y_train = train['sentiment']
        self.y_test = test['sentiment']

        # -- dataset preprocessing -- #
        train_tokens = self.tokenize_data(train_txt, 'training')
        test_tokens = self.tokenize_data(test_txt, 'testing')

        # -- vectorize training/testing data -- #
        train_vectors = self.average_feature_vectors(train_tokens, 'training')
        test_vectors = self.average_feature_vectors(test_tokens, 'testing')

        # vectorized features
        self.X_train = self.remove_nan(train_vectors)
        self.X_test = self.remove_nan(test_vectors)

        info('Done loading and vectorizing data.')
        info("--- Sentiment CLASSIFIERS ---")
        info("fitting ... ")

        # classifiers to use
        classifiers = [
            RandomForestClassifier(n_estimators=100),
            SGDClassifier(loss='log', penalty='l1'),
            LinearSVC(C=1e1),
            NuSVC(),
            LogisticRegressionCV(solver='liblinear'),
            GaussianNB(),
        ]

        self.accuracies = {}

        # RUN classifiers
        for c in classifiers:
            self.classify(c, detailed, plot_roc)

        avg_f1 = 0
        info('results ...')
        for k, v in self.accuracies.items():
            string = '\tMacAvg. {:.2f}% F1. {:.2f}% P. {:.2f} R. {:.2f} : {}'
            print(string.format(v[0] * 100, v[1] * 100, v[2] * 100, v[3] * 100, k))
            avg_f1 += float(v[1])

        #print('OVERALL avg F1 test {:.2f}%'.format((avg_f1 / len(self.accuracies)) * 100))
        info("DONE!")

    @staticmethod
    def load_vectors(model_name, binary=True):
        """load the pre-trained embedding model"""
        if binary:
            w2v_model = KeyedVectors.load_word2vec_format(model_name, binary=True)
        else:
            w2v_model = KeyedVectors.load(model_name)

        w2v_model.init_sims(replace=True)  # to save memory
        vocab, vector_dim = w2v_model.syn0.shape
        return w2v_model, vector_dim


    @staticmethod
    def tokenize(text):
        """
        :param text: a paragraph string
        :return: a list of words
        """

        try:
            try:
                txt = unicode(text, 'utf-8')  # py2
            except NameError:
                txt = text  # py3
            words = wordpunct_tokenize(txt)
            length = len(words)
        except TypeError:
            words, length = ['NA'], 0

        return words, length

    def tokenize_data(self, examples_txt, type_='NaN'):
        tokens = []
        info('Tokenizing the {} dataset ..'.format(type_))
        total_tokens = []
        for txt in examples_txt:
            words, num = self.tokenize(txt)
            tokens.append(words)
            total_tokens.append(num)
        info(' ... total {} {} tokens.'.format(sum(total_tokens), type_))
        return tokens

    def feature(self, words):
        """average words' vectors"""

        feature_vec = np.zeros((self.dimension,), dtype="float32")
        retrieved_words = 0
        for token in words:
            try:
                feature_vec = np.add(feature_vec, self.embeddings[token])
                retrieved_words += 1
            except KeyError:
                pass  # if a word is not in the embeddings' vocabulary discard it

        np.seterr(divide='ignore', invalid='ignore')
        feature_vec = np.divide(feature_vec, retrieved_words)

        return feature_vec

    def average_feature_vectors(self, examples, type_='NaN'):
        """
        :param examples: a list of lists (each list contains words) e.g. [['hi','do'], ['you','see'], ... ]
        :param type_: (optional) type of examples text e.g. train / test
        :return: the average word vector of each list
        """

        feature_vectors = np.zeros((len(examples), self.dimension), dtype="float32")
        info("Vectorizing {} tokens ..".format(type_))
        for i, example in enumerate(examples):
            feature_vectors[i] = self.feature(example)

        info(" ... total {} {}".format(len(feature_vectors), type_))

        return feature_vectors

    def classify(self, classifier=None, info_=False, plot_roc=False):

        classifier_name = classifier.__class__.__name__

        if info_:
            info('fitting data ...')
            info('\n\ncreated \n\n{}'.format(classifier))

        classifier.fit(self.X_train, self.y_train)
        pscore = classifier.score(self.X_test, self.y_test)

        if info_:
            info('\n\n\t{}() ACCURACY: {}\n'.format(classifier_name, pscore))

        # F1 score
        y_pred = classifier.predict(self.X_test)
        f1_score = metrics.f1_score(self.y_test, y_pred)

        # macro accuracy (macro average)
        macc = metrics.f1_score(self.y_test, y_pred, pos_label=None, average='macro')

        # precision and recall
        recall = metrics.recall_score(self.y_test, y_pred)
        precision = metrics.precision_score(self.y_test, y_pred)

        results = [macc, f1_score, precision, recall]
        self.accuracies[classifier_name] = results

        # prediction
        negative = len(classifier.predict(self.X_test)[classifier.predict(self.X_test) == 0])
        positive = len(classifier.predict(self.X_test)[classifier.predict(self.X_test) == 1])

        if plot_roc:
            info('plotting roc of ... {}'.format(classifier_name))
            self.plot_auc(classifier, classifier_name, negative, positive)

    def plot_auc(self, estimator, estimator_name, neg, pos):
        try:
            classifier_probas = estimator.decision_function(self.X_test)
        except AttributeError:
            classifier_probas = estimator.predict_proba(self.X_test)[:, 1]

        false_positive_r, true_positive_r, thresholds = metrics.roc_curve(self.y_test, classifier_probas)
        roc_auc = metrics.auc(false_positive_r, true_positive_r)

        label = '{:.1f}% neg:{} pos:{} {}'.format(roc_auc * 100, neg, pos, estimator_name)
        plt.plot(false_positive_r, true_positive_r, label=label)
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([-0.05, 1.0])
        plt.ylim([0.0, 1.05])
        plt.title('ROC score(s)')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.legend(loc='lower right', prop={'size': 10})
        plt.savefig("ROC.png", dpi=300, bbox_inches='tight')
        plt.grid()

    @staticmethod
    def remove_nan(x):
        """remove NaN values from data vectors"""
        imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
        x_clean = imp.fit_transform(x)
        return x_clean
if __name__ == "__main__":
    #parser = argparse.ArgumentParser()
    #parser.add_argument("embeddings/arabic-news.bin", help="path a pre-trained vectors model.")
    #parser.add_argument("datasets/LABR-book-reviews.csv", help="path a labeled (0/1) sentiment dataset.")

    #args = parser.parse_args()
    #vec = args.vectors

    # vectors file
    embeddings_path = "arabic-news.bin"
    # dataset file
    dataset_path =  "datasets/Aziz/datasets/astd-artwitter.csv"

    # run
    ArSentiment(train, test, embeddings_path, dataset_path, plot_roc=False)


[2018-08-16 20:37:20,419] INFO: loading projection weights from arabic-news.bin
[2018-08-16 20:37:22,976] INFO: loaded (159175, 300) matrix from arabic-news.bin
[2018-08-16 20:37:22,976] INFO: precomputing L2-norms of word weight vectors
C:\Users\aas3n17\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:77: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
[2018-08-16 20:37:24,936] INFO: Tokenizing the training dataset ..
[2018-08-16 20:37:24,951] INFO:  ... total 15995 training tokens.
[2018-08-16 20:37:24,951] INFO: Tokenizing the testing dataset ..
[2018-08-16 20:37:24,951] INFO:  ... total 1841 testing tokens.
[2018-08-16 20:37:24,951] INFO: Vectorizing training tokens ..
[2018-08-16 20:37:25,040] INFO:  ... total 1755 training
[2018-08-16 20:37:25,040] INFO: Vectorizing testing tokens ..
[2018-08-16 20:37:25,058] INFO:  ... total 196 testing
[2018-08-16 20:37:25,078] INFO: Done loading and vecto

	MacAvg. 82.59% F1. 81.52% P. 83.33 R. 79.79 : RandomForestClassifier
	MacAvg. 84.18% F1. 84.26% P. 80.58 R. 88.30 : SGDClassifier
	MacAvg. 85.19% F1. 84.66% P. 84.21 R. 85.11 : LinearSVC
	MacAvg. 83.16% F1. 82.90% P. 80.81 R. 85.11 : NuSVC
	MacAvg. 83.67% F1. 83.51% P. 81.00 R. 86.17 : LogisticRegressionCV
	MacAvg. 65.97% F1. 69.44% P. 61.48 R. 79.79 : GaussianNB


In [163]:
#LSTM
def get_stop_words():
    path = "Arabic_stop_word.txt"
    stop_words = []
    with codecs.open(path, "r", encoding="utf-8", errors="ignore") as myfile:
        stop_words = myfile.readlines()
    stop_words = [word.strip() for word in stop_words]
    return stop_words

def list_seq(train, test):
    stop_words = get_stop_words()
    list_sentences_train = train["txt"]
    list_sentences_test = test["txt"]
    max_features = 8000
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(list_sentences_train))
    list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
    list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
    return list_tokenized_train, list_tokenized_test



def calculate_accuracy1(X_test, y_test):
        # F1 score
        y_pred = model.predict(X_test)
        y_pred=y_pred.round()
        f1_score = metrics.f1_score(y_test, y_pred)

        # macro accuracy (macro average)
        macc = metrics.f1_score(y_test, y_pred, pos_label=None, average='macro')

        # precision and recall
        recall = metrics.recall_score(y_test, y_pred)
        precision = metrics.precision_score(y_test, y_pred)
        accu=metrics.accuracy_score(y_test, y_pred)

        results = [accu,macc, f1_score, precision, recall]
        print("macc, f1_score, precision, recall", results)


def model_history(train, test,list_tokenized_train, list_tokenized_test, epochs, batch_size):
    list_classes = ["sentiment"]
    y = train['sentiment']
    maxlen = 85 # you can tune
    X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
    X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
    inp = Input(shape=(maxlen, )) 
    max_features = 8000

    embed_size = 300 #you can tune
    x = Embedding(max_features, embed_size)(inp)
    x = LSTM(30, return_sequences=True,name='lstm_layer')(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.5)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
    history=model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.1)
    return history, model, X_te


list_tokenized_train, list_tokenized_test=list_seq(train, test)
    #word_dis(list_tokenized_train)
history, model, X_te= model_history(train, test,list_tokenized_train, list_tokenized_test,30,32)


Train on 1579 samples, validate on 176 samples
Epoch 1/30
1579/1579 [==============================] - 6s 4ms/step - loss: 0.6838 - acc: 0.5795 - val_loss: 0.6634 - val_acc: 0.6364
Epoch 2/30
1579/1579 [==============================] - 4s 3ms/step - loss: 0.5706 - acc: 0.8132 - val_loss: 0.4514 - val_acc: 0.8580
Epoch 3/30
1579/1579 [==============================] - 4s 3ms/step - loss: 0.2461 - acc: 0.9360 - val_loss: 0.3206 - val_acc: 0.8636
Epoch 4/30
1579/1579 [==============================] - 4s 3ms/step - loss: 0.0875 - acc: 0.9880 - val_loss: 0.3528 - val_acc: 0.8807
Epoch 5/30
1579/1579 [==============================] - 4s 3ms/step - loss: 0.0473 - acc: 0.9956 - val_loss: 0.4062 - val_acc: 0.8750
Epoch 6/30
1579/1579 [==============================] - 4s 3ms/step - loss: 0.0322 - acc: 0.9962 - val_loss: 0.4529 - val_acc: 0.8523
Epoch 7/30
1579/1579 [==============================] - 4s 3ms/step - loss: 0.0263 - acc: 0.9968 - val_loss: 0.4419 - val_acc: 0.8580
Epoch 8/30
1579

In [164]:
y_true = test['sentiment']
calculate_accuracy1(X_te, y_true)

macc, f1_score, precision, recall [0.8163265306122449, 0.815847165674914, 0.8252427184466019, 0.7589285714285714, 0.9042553191489362]
